In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision 
from torch.utils.data import DataLoader
from tqdm import tqdm

In [28]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.conv3 = nn.Conv2d(
            in_channels=16,
            out_channels=1,
            kernel_size=(7, 7),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
input_channels=1
num_classes=10
lr=0.001
batch_size=64
num_epochs=10

In [31]:
train_dataset= datasets.MNIST(root='../datasets/', 
                             train= True,
                             download=True,
                             transform=transforms.ToTensor())
test_dataset= datasets.MNIST(root='../datasets/', 
                             train= False,
                             download=True,
                             transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
test_loader  = DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle= True)


In [32]:
model = CNN().to(device)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [34]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.to(device=device)
        targets = targets.to(device=device)
        #data = data.reshape(data.shape[0], -1) #flatten 
        
        #forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()

100%|███████████████████████████████████████████████████████████████████████████████| 938/938 [00:05<00:00, 159.30it/s]


In [35]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

100%|███████████████████████████████████████████████████████████████████████████████| 938/938 [00:04<00:00, 197.42it/s]


Accuracy on training set: 99.08


100%|███████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 194.40it/s]

Accuracy on test set: 98.51
